In [ ]:
class Grammar:
    @staticmethod
    def parseLine(line):
        return [ value.strip() for value in line.strip().split('=')[1].strip()[1:-1].strip().split(',')]
    
    @staticmethod
    def readFrom(fileName):
        with open(fileName) as file: 
            N = Grammar.parseLine(file.readline())
            E = Grammar.parseLine(file.readline())
            S = file.readline().split('=')[1].strip()
            P = Grammar.parseRules(Grammar.parseLine(''.join([line for line in file])))
            
            return Grammar(N, E, P, S)
        
    @staticmethod        
    def parseRules(rules):
        result = []
        
        for rule in rules:
            lhs, rhs = rule.split('->')
            lhs = lhs.strip()
            rhs = [ value.strip() for value in rhs.split('|')]
            
            for value in rhs: 
                result.append((lhs, value))
        
        return result 
    
    @staticmethod
    def fromFiniteAutomata(fa):
        N = fa.Q
        E = fa.E 
        S = fa.q0
        P = []
        
        for transition in fa.S: 
            lhs, state2 = transition
            state1, route = lhs
            
            P.append((state1, route + state2))
            
            if state2 in fa.F: 
                P.append((state1, route))
                
        return Grammar(N, E, P, S)
    
    def __init__(self, N, E, P, S):
        self.N = N 
        self.E = E
        self.P = P
        self.S = S
        
    def isNonTerminal(self, char):
        return char in self.N
    
    def isTerminal(self, char):
        return char in self.E 
    
    def isRegular(self):
        usedInRhs = dict() 
        notAllowedInRhs = list() 
        
        for rule in self.P: 
            lhs, rhs = rule
            hasTerminal = False 
            hasNonTerminal = False
            for char in rhs: 
                if self.isNonTerminal(char): 
                    usedInRhs[char] = True
                    hasNonTerminal = True
                elif self.isTerminal(char): 
                    if hasNonTerminal: 
                        return False
                    hasTerminal = True 
                if char == 'E': 
                    notAllowedInRhs.append(lhs)
                    
            if hasNonTerminal and not hasTerminal: 
                return False
        
        for char in notAllowedInRhs: 
            if char in usedInRhs: 
                return False 
            
        return True
    
    def __str__(self):
        return 'N = { ' + ', '.join(self.N) + ' }\n' \
             + 'E = { ' + ', '.join(self.E) + ' }\n' \
             + 'P = { ' + ', '.join([' -> '.join(rule) for rule in self.P]) + ' }\n' \
             + 'S = ' + str(self.S) + '\n' 

In [ ]:
grammar = Grammar.readFrom('rg1.txt') 
print(grammar)
print(grammar.isRegular())

In [ ]:
with open(grammarFileName) as file: 
    for line in file: 
        print(line)

In [ ]:
class FiniteAutomata: 
    @staticmethod
    def parseLine(line):
        return [ value.strip() for value in line.strip().split('=')[1].strip()[1:-1].strip().split(',')]
    
    @staticmethod
    def readFrom(fileName):
        with open(fileName) as file: 
            Q = FiniteAutomata.parseLine(file.readline())
            E = FiniteAutomata.parseLine(file.readline())
            q0 = file.readline().split('=')[1].strip()            
            F = FiniteAutomata.parseLine(file.readline())
            
            S = FiniteAutomata.parseTransitions(FiniteAutomata.parseLine(''.join([line for line in file])))
            
            return FiniteAutomata(Q, E, S, q0, F)
        
    
    @staticmethod
    def parseTransitions(parts):
        result = []
        transitions = []
        index = 0 
        
        while index < len(parts): 
            transitions.append(parts[index] + ',' + parts[index + 1])
            index += 2
            
        for transition in transitions:
            lhs, rhs = transition.split('->')
            state2 = rhs.strip()
            state1, route = [ value.strip() for value in lhs.strip()[1:-1].split(',') ]
            
            result.append(((state1, route), state2))
        
        return result 
    
    @staticmethod
    def fromRegularGrammar(rg):
        Q = rg.N + ['K']
        E = rg.E
        q0 = rg.S
        F = ['K']
        
        S = [] 
        
        for production in rg.P: 
            state2 = 'K'
            state1, rhs = production
            if state1 == q0 and rhs[0] == 'E': 
                F.append(q0)
                continue 
                
            route = rhs[0]
            
            if len(rhs) == 2: 
                state2 = rhs[1]
            
            S.append(((state1, route), state2))
            
        return FiniteAutomata(Q, E, S, q0, F)
    
    def __init__(self, Q, E, S, q0, F):
        self.Q = Q
        self.E = E
        self.S = S
        self.q0 = q0
        self.F = F
    
    
    def __str__(self):
        return 'Q = { ' + ', '.join(self.Q) + ' }\n' \
             + 'E = { ' + ', '.join(self.E) + ' }\n' \
             + 'F = { ' + ', '.join(self.F) + ' }\n' \
             + 'S = { ' + ', '.join([' -> '.join([str(part) for part in trans]) for trans in self.S]) + ' }\n' \
             + 'q0 = ' + str(self.q0) + '\n' 

In [ ]:
fa = FiniteAutomata.readFrom('fa1.txt')
print(fa)

In [ ]:
z = Grammar.fromFiniteAutomata(fa)
print(z.isRegular())
print(z)

In [ ]:
q = FiniteAutomata.fromRegularGrammar(z)
print(q)

In [ ]:
x = Grammar.fromFiniteAutomata(q)
print(x.isRegular())
print(x)

In [ ]:
y = FiniteAutomata.fromRegularGrammar(x)
print(y)